In [1]:
import gc
import torch
import sys

gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install rouge_score
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [3]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
from datasets import Dataset



import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset

)

from transformers import RobertaTokenizerFast
from transformers import EncoderDecoderModel
from dataclasses import dataclass, field
from typing import Optional
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
from tabulate import tabulate


In [5]:
model_checkpoints = "/aiffel/aiffel/Korean_Conversation_Summary/MLM_pretrain_5ep/checkpoint-21500"#"gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [6]:
# train_cat = pd.read_csv('data/train_total.csv')
# val_cat = pd.read_csv('data/val_total.csv')

In [7]:
# train_cat = train_cat.drop(['Id','Text', 'Summary'], axis=1)
# val_cat = val_cat.drop(['Id','Text', 'Summary'], axis=1)

In [8]:
# train_cat

In [9]:
# train_cat.to_csv('data/train_category.csv', index = False)
# val_cat.to_csv('data/val_category.csv', index = False)

In [10]:
# train_cat = pd.read_csv('data/train_category.csv')
# val_cat = pd.read_csv('data/val_category.csv')
# val_cat.head()

In [11]:
train_category_path = "data/train_category.csv"
val_category_path = "data/val_category.csv"

with open(train_category_path, encoding="utf-8") as f:
            train_category = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(val_category_path, encoding="utf-8") as f:
            val_category = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

In [12]:
train_textfile_path = "data/train_text.csv"
train_summaryfile_path = "data/train_summary.csv"

with open(train_textfile_path, encoding="utf-8") as f:
            train_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(train_summaryfile_path, encoding="utf-8") as f:
            train_sumlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]          

In [13]:
val_textfile_path = "data/val_text.csv"
val_summaryfile_path = "data/val_summary.csv"

with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

with open(val_summaryfile_path, encoding="utf-8") as f:
            val_sumlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]          

In [14]:
del train_textlines[0]
del val_textlines[0]
del train_category[0]
del val_category[0]
del val_sumlines[0]
del train_sumlines[0]

In [15]:
for i in range(len(val_textlines)):
    temp_cat = "#"+val_category[i]+"# "
    val_textlines[i] = temp_cat+val_textlines[i]
   

In [16]:
for i in range(len(train_textlines)):
    temp_cat = "#"+train_category[i]+"# "
    train_textlines[i] = temp_cat+train_textlines[i]

In [17]:
train_df = pd.DataFrame(zip(train_textlines, train_sumlines), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_textlines, val_sumlines), columns=['Text', 'Summary'])

In [18]:
train_df.head()

,Text,Summary
0,#상거래(쇼핑)# 그럼 날짜는 가격 큰 변동 없으면 6 28 7 13로 확정할까 우...,비행기 표 가격에 대해 이야기하며 특가 이벤트를 기다리고 있다
1,#상거래(쇼핑)# kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마...,비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다
2,#상거래(쇼핑)# 아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 ...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
3,#상거래(쇼핑)# 칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ 아 ...,칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다
4,#상거래(쇼핑)# 잠도안오네ㅐ얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보...,잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다


In [19]:
# DF > data Set으로 전환
train_len = len(train_df) // 4
train_data = Dataset.from_pandas(train_df[:train_len]) 
val_len = len(val_df) // 2
val_data = Dataset.from_pandas(val_df[:val_len])
test_data=Dataset.from_pandas(val_df[val_len:])

In [20]:
test_data[0]

{'Text': '#여가 생활# 웹툰볼그야ㅋㅋ 안자면바야지 곧유료화되는것들 먼저봐야지 이런영웅은싫어 괜찮다던데 17일인가유료화 그게머지 첨들어보는데 나도몰랏는데 인기글에서봤어 웅ㅋㅋㅋ 보고 재밋으면말해준다',
 'Summary': '곧 유료화되는 웹툰을 보고 재미있으면 말해주겠다'}

In [21]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 69998)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 17502)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 17502)


In [22]:
max_input = 128
max_target = 128
batch_size = 4

#"digit82/kobart-summarization"

In [23]:
def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['Text']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['Summary'], max_length=max_target, padding='max_length', truncation=True)
    
  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [24]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [26]:
# # set special tokens
# #from transformers import EncoderDecoderConfig
# model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
# model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 128 # 256은 쿠다 메모리 오류 생김
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 2
model.config.length_penalty = 2.0
model.config.num_beams = 2


In [27]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    rouge_outputL = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid
    

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
        
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4), 
        
        "rougeL_precision": round(rouge_outputL.precision, 4),
        "rougeL_recall": round(rouge_outputL.recall, 4),
        "rougeL_fmeasure": round(rouge_outputL.fmeasure, 4),
    }

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="meta_test",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=16,
    learning_rate=3e-05,
    warmup_steps=2000,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=2000,
    save_total_limit=3,
)

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?
"""
DataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . 
또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.
tokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.
"""

'\nDataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . \n또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.\ntokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.\n'

In [30]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 69998
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4375
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2000,2.517200
4000,1.660800


Saving model checkpoint to meta_test/checkpoint-500
Configuration saved in meta_test/checkpoint-500/config.json
Model weights saved in meta_test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in meta_test/checkpoint-500/tokenizer_config.json
Special tokens file saved in meta_test/checkpoint-500/special_tokens_map.json
Saving model checkpoint to meta_test/checkpoint-1000
Configuration saved in meta_test/checkpoint-1000/config.json
Model weights saved in meta_test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in meta_test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in meta_test/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to meta_test/checkpoint-1500
Configuration saved in meta_test/checkpoint-1500/config.json
Model weights saved in meta_test/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in meta_test/checkpoint-1500/tokenizer_config.json
Special tokens file saved in meta_test/checkpoint-1500/special_toke

TrainOutput(global_step=4375, training_loss=2.051552176339286, metrics={'train_runtime': 2755.2332, 'train_samples_per_second': 25.405, 'train_steps_per_second': 1.588, 'total_flos': 5335041575485440.0, 'train_loss': 2.051552176339286, 'epoch': 1.0})

In [32]:
len(test_data)

17502

In [1]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)# 여기에 기본 kobart가져오기?
import random
from random import randrange
ck_num = len(test_data)
test_samples = test_data.select(range(0, ck_num, 20))# 0, len(test_data), 200

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1] # 여기에 체크포인트 가져오기 
# 연구해봐야한다.

NameError: name 'AutoModelForSeq2SeqLM' is not defined

In [ ]:
for i in range(len(summaries_after_tuning)):
    print('idx_{} '.format(i))
    print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()
    

메타 데이터를 pretrain에 적용할지 -> fine-tuning에 적용해야하는 것 같다. 그런데 이걸 eos 처럼 넣어야 할지 잘모르겠다..
